In [176]:
import pandas as pd
import numpy as np

# =====================================================
# CONFIGURAÇÕES
# =====================================================
DATE_START = "2025-01-01"
DATE_END   = "2025-12-01"

PATH_LANCAMENTOS = r"C:\Users\dorag\Downloads\Lancamentos_CGL_2025.xlsx"
PATH_PLANO = r"C:\Users\dorag\Downloads\Plano_Contas_CGL.xlsx"

# =====================================================
# CARGA
# =====================================================

def carregar_base(path):
    return pd.read_excel(path, engine="openpyxl")


def carregar_plano_contas(path):
    return pd.read_excel(path, engine="openpyxl")


# =====================================================
# PRÉ-PROCESSAMENTO
# =====================================================

def filtrar_periodo(df, date_start, date_end):
    df = df.loc[:, ["Data", "Conta Débito", "Conta Crédito", "Valor", "Descrição Histórico"]].copy()
    df.loc[:, "Data"] = pd.to_datetime(df["Data"])
    return df.loc[
        (df["Data"] >= date_start) &
        (df["Data"] < date_end)
    ].copy()


# =====================================================
# PARTIDA DOBRADA
# =====================================================

def normalizar_partida_dobrada(df):
    df_debito = df.loc[:, ["Data", "Conta Débito", "Valor", "Descrição Histórico"]].copy()
    df_debito.loc[:, "Conta Completa"] = df_debito["Conta Débito"]
    df_debito.loc[:, "D/C"] = "D"
    df_debito.loc[:, "Valor"] = -df_debito["Valor"]

    df_credito = df.loc[:, ["Data", "Conta Crédito", "Valor", "Descrição Histórico"]].copy()
    df_credito.loc[:, "Conta Completa"] = df_credito["Conta Crédito"]
    df_credito.loc[:, "D/C"] = "C"

    return pd.concat(
        [
            df_debito[["Data", "Conta Completa", "D/C", "Valor", "Descrição Histórico"]],
            df_credito[["Data", "Conta Completa", "D/C", "Valor", "Descrição Histórico"]],
        ],
        ignore_index=True
    )


def quebrar_conta(df):
    df = df.copy()
    df.loc[:, "Conta Código"] = df["Conta Completa"].str.extract(r"^(\d+)")
    df.loc[:, "Conta Nome"] = df["Conta Completa"].str.extract(r"-\s*(.*)")
    df.loc[:, "valor_abs"] = df["Valor"].abs().round(2)
    return df


# =====================================================
# DETECÇÃO HIERÁRQUICA DO PLANO
# =====================================================

def detectar_conta_pai(df_plano, palavras, grupo):
    return df_plano.loc[
        (df_plano["Analítica"] == False) &
        (df_plano["Grupo Conta"] == grupo) &
        (df_plano["Descrição"].str.upper().str.contains("|".join(palavras)))
    ]


def marcar_hierarquia(df_plano, pais, tipo):
    mapa = {}
    for _, pai in pais.iterrows():
        prefixo = str(pai["Conta"])
        filhos = df_plano.loc[
            (df_plano["Conta"].astype(str).str.startswith(prefixo)) &
            (df_plano["Analítica"] == True)
        ]
        for _, f in filhos.iterrows():
            mapa[str(f["Código Reduzido"])] = tipo
    return mapa


def gerar_mapa_plano_contas(df_plano):
    mapa = {}

    mapa.update(marcar_hierarquia(
        df_plano,
        detectar_conta_pai(df_plano, ["CLIENTE", "CONTAS A RECEBER"], 1),
        "CLIENTE"
    ))

    mapa.update(marcar_hierarquia(
        df_plano,
        detectar_conta_pai(df_plano, ["CAIXA", "BANCO", "BANCOS", "DISPONIVEL"], 1),
        "FINANCEIRO"
    ))

    mapa.update(marcar_hierarquia(
        df_plano,
        detectar_conta_pai(df_plano, ["FORNECEDOR", "FORNECEDORES", "CONTAS A PAGAR"], 2),
        "FORNECEDOR"
    ))

    receitas = df_plano.loc[(df_plano["Grupo Conta"] == 3) & (df_plano["Analítica"] == True)]
    for _, r in receitas.iterrows():
        mapa[str(r["Código Reduzido"])] = "RECEITA"

    despesas = df_plano.loc[(df_plano["Grupo Conta"] == 4) & (df_plano["Analítica"] == True)]
    for _, d in despesas.iterrows():
        mapa[str(d["Código Reduzido"])] = "DESPESA"

    despesas = df_plano.loc[(df_plano["Grupo Conta"] == 5) & (df_plano["Analítica"] == True)]
    for _, d in despesas.iterrows():
        mapa[str(d["Código Reduzido"])] = "PATRIMONIO"
    
    return mapa


def classificar_contas_por_plano(df, mapa):
    df = df.copy()
    df.loc[:, "tipo_conta"] = df["Conta Código"].astype(str).map(mapa).fillna("OUTRO")
    return df


# =====================================================
# IDENTIFICAÇÃO DE CLIENTE (NOVA, CORRETA)
# =====================================================

def identificar_cliente(df):
    df = df.copy()
    df.loc[:, "Cliente"] = np.where(
        df["tipo_conta"] == "CLIENTE",
        df["Conta Nome"].str.upper().str.strip(),
        pd.NA
    )
    return df


# =====================================================
# CONCILIAÇÃO (INALTERADA)
# =====================================================

def conciliar_linhas(df):
    df = df.sort_values("Data").reset_index(drop=True).copy()
    df.loc[:, "status_conciliacao"] = "NAO CONCILIADO"
    df.loc[:, "id_conciliacao"] = pd.NA

    conciliacao_id = 1

    for idx, row in df.iterrows():
        if df.at[idx, "status_conciliacao"] == "CONCILIADO":
            continue

        valor = row["valor_abs"]
        cliente = row["Cliente"]
        data = row["Data"]
        tipo = row["tipo_conta"]
        dc = row["D/C"]

        if tipo == "RECEITA" and dc == "C":
            candidatos = df.loc[
                (df.index != idx) &
                (df["tipo_conta"] == "FINANCEIRO") &
                (df["D/C"] == "D") &
                (df["valor_abs"] == valor) &
                (df["Cliente"] == cliente) &
                (df["Data"] >= data) &
                (df["status_conciliacao"] == "NAO CONCILIADO")
            ]

        elif tipo == "CLIENTE" and dc == "D":
            candidatos = df.loc[
                (df.index != idx) &
                (df["tipo_conta"] == "CLIENTE") &
                (df["D/C"] == "C") &
                (df["valor_abs"] == valor) &
                (df["Cliente"] == cliente) &
                (df["Data"] >= data) &
                (df["status_conciliacao"] == "NAO CONCILIADO")
            ]
        else:
            continue

        if not candidatos.empty:
            idx_par = candidatos.index[0]
            df.at[idx, "status_conciliacao"] = "CONCILIADO"
            df.at[idx_par, "status_conciliacao"] = "CONCILIADO"
            df.at[idx, "id_conciliacao"] = conciliacao_id
            df.at[idx_par, "id_conciliacao"] = conciliacao_id
            conciliacao_id += 1

    return df


# =====================================================
# STATUS FINAL
# =====================================================

def classificar_status(df):
    df = df.copy()

    def definir(row):
        if row["status_conciliacao"] == "CONCILIADO":
            return "CONCILIADO"
        if row["tipo_conta"] == "RECEITA" and row["D/C"] == "C":
            return "NF EM ABERTO"
        if row["tipo_conta"] == "CLIENTE" and row["D/C"] == "D":
            return "NF EM ABERTO"
        if row["tipo_conta"] == "CLIENTE" and row["D/C"] == "C":
            return "RECEBIDO SEM NF"
        if row["tipo_conta"] == "FINANCEIRO" and row["D/C"] == "D":
            return "RECEBIDO SEM NF"
        return "OUTRO"

    df.loc[:, "status_conciliacao"] = df.apply(definir, axis=1)
    return df


# =====================================================
# PIPELINE FINAL
# =====================================================

def executar_conciliacao():
    df = carregar_base(PATH_LANCAMENTOS)
    df = filtrar_periodo(df, DATE_START, DATE_END)
    df = normalizar_partida_dobrada(df)
    df = quebrar_conta(df)

    df_plano = carregar_plano_contas(PATH_PLANO)
    mapa = gerar_mapa_plano_contas(df_plano)

    df = classificar_contas_por_plano(df, mapa)
    df = identificar_cliente(df)
    df = conciliar_linhas(df)
    df = classificar_status(df)

    return df[
        ["Data", "Cliente", "Conta Código", "Conta Nome", "D/C",
         "tipo_conta", "status_conciliacao", "Valor", "Descrição Histórico"]
    ]


# =====================================================
# EXECUÇÃO
# =====================================================

df_final = executar_conciliacao()
df_final


,Data,Cliente,Conta Código,Conta Nome,D/C,tipo_conta,status_conciliacao,Valor,Descrição Histórico
0,2025-01-01,<NA>,80003,EI Advanced Servicos em INformatica,C,FORNECEDOR,OUTRO,418.95,vr ref nfse 11354
1,2025-01-01,<NA>,16320,Assistência Médica,D,DESPESA,OUTRO,-80.00,vr ref nfse 13628666
2,2025-01-01,<NA>,16652,Descontos Concedidos,D,DESPESA,OUTRO,-20.00,vr ref desconto cgr
3,2025-01-01,CAROLINA RIBEIRO DINIZ,70067,CAROLINA RIBEIRO DINIZ,C,CLIENTE,RECEBIDO SEM NF,738.90,vr ref receb.
4,2025-01-01,<NA>,10101,Caixa,D,FINANCEIRO,RECEBIDO SEM NF,-738.90,vr ref receb.
...,...,...,...,...,...,...,...,...,...
2835,2025-11-30,<NA>,16732,Aplicações Financeiras,C,RECEITA,NF EM ABERTO,0.99,vr ref rendimento aplicacao
2836,2025-11-30,<NA>,16301,Salários,D,DESPESA,OUTRO,-1173.33,vr.ref. saldo salario rescisao
2837,2025-11-30,<NA>,16732,Aplicações Financeiras,C,RECEITA,NF EM ABERTO,4138.27,vr ref rendimento aplicacao
2838,2025-11-30,<NA>,NaN,NaN,D,OUTRO,OUTRO,-130.00,NaN


In [177]:
# CLIENTE_ALVO = 'VERTEX INDUSTRIA E COMERCIO DE EMBALAGENS LTDA' 

# df_filtrado = df_final[df_final['Cliente'] == CLIENTE_ALVO].copy().reset_index()

# df_filtrado

In [178]:
# df_final.to_excel(r"C:\Users\dorag\Downloads\Conciliação.xlsx", index=False)

In [179]:
CONTA_ALVO = 'CLIENTE'

df_filtrado = df_final[df_final['tipo_conta'] == CONTA_ALVO].copy()

df_filtrado.groupby("Conta Nome")["Valor"].sum()

# df_final["Valor"].sum()

Conta Nome
49.101.166 VALDIRENE MARIA DOS SANTOS                               0.00
49.447.820 ILDEU DA SILVA ANTONIAZZI                                0.00
50.983.463 NATALIA DE OLIVEIRA GOMES                             -300.00
53.474.561 CINTIA ALVES MARCHI SERPA                                0.00
57.115.523 RENATA CURI ZOFFOLI                                   -200.00
AA Comercio de Cabelos e Produtos                               -9529.80
ASMYSEG CORRETORA DE SEGUROS  LTDA                              -7775.30
AURA SOLUÃOES INDUSTRIAIS LYDA                                      0.00
Adriana Nery da Silva Passos                                       -1.00
BG LOCACAO E TRANSPORTES LTDA                                    3323.40
Bave Locadora de Bens Moveis Ltda                               -1728.10
CAROLINA RIBEIRO DINIZ                                          -2853.00
CENTRO MUNICIPAL PINOQUIO                                           0.00
CGR Equipamentos e Paineis Eletricos Ltd